In [1]:
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [2]:
#The dataframe
untouched_csv = pd.read_csv('CSV/RAW_data.csv', sep=',')

In [3]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)
untouched_csv.infer_objects(copy=False)

C:\Users\Krupam\AppData\Local\Temp\ipykernel_5752\3870762404.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)


,Date,Hr,Mainland Air Temp,Humidity (%),Direction (A),Wind Speed (A),Gusting,Atmospheric Pressure (IN),Precipitation Rate,Bay Temp,Salinity,LBI Air Temp,Ocean Temp
0,6/1/2025,0,53.3,88.0,270.0,6.8,12.1,29.54,0.0,17.52,28.34,56.1,14.5
1,6/1/2025,1,52.2,86.0,270.0,5.5,12.5,29.55,0.0,17.43,27.89,55.4,14.5
2,6/1/2025,2,50.7,83.0,247.5,4.5,9.3,29.57,0.0,17.42,27.90,55.1,14.4
3,6/1/2025,3,50.1,79.0,225.0,2.6,4.0,29.59,0.0,17.34,28.57,53.9,14.4
4,6/1/2025,4,47.7,86.0,247.5,3.9,6.2,29.60,0.0,17.23,28.20,53.9,14.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,9/23/2025,20,70.2,88.0,0.0,6.0,9.1,29.96,0.0,21.84,30.30,72.3,21.3
2757,9/23/2025,21,69.6,89.0,202.5,4.2,6.4,29.96,0.0,21.61,29.96,72.6,21.3
2758,9/23/2025,22,70.5,83.0,202.5,4.5,7.5,29.96,0.0,22.02,30.06,73.4,21.3
2759,9/23/2025,23,70.7,84.0,225.0,3.8,5.3,29.97,0.0,22.09,30.03,73.2,21.3


In [4]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [5]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [6]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv['Onshore'] = untouched_csv['Onshore'].astype(int)



In [7]:
#thresholds
ocean_thresh = 1.0
wind_thresh = 7.0

In [8]:
#average of two lowest ocean points in a day
untouched_csv['ocean_min1'] = untouched_csv['Ocean Temp'].rolling(24).min()
untouched_csv['ocean_min2'] = (
    untouched_csv['Ocean Temp'].rolling(24).apply(lambda x: np.sort(x)[1], raw=False)
)
untouched_csv['ocean_min'] = (untouched_csv['ocean_min1']+untouched_csv['ocean_min2'])/2
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [9]:
#Check if its a upwelling direction
upwell_wind = [135, 165, 180, 270]
untouched_csv['upwell_wind'] = untouched_csv['Direction (A)'].isin(upwell_wind)
untouched_csv['upwell_wind'] = untouched_csv['upwell_wind'].astype(int)

In [10]:
#bools:
#   There are two tide cycles in a day, if you take the two lowest values and average them together. 
#   Taking the absolute value of the difference of the low values and the current temperature and if this is bigger than the threshold the Ocean boolean is true. 
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv['upwell_wind'] = (untouched_csv['upwell_wind'] == 1).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv['ocean_min'] - untouched_csv['Ocean Temp']) > ocean_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)

In [11]:
#drop the colomns
untouched_csv.drop(['ocean_bool', 'big_wind', 'ocean_min1', 'ocean_min2', 'upwell_wind', 'ocean_min'], axis=1, inplace=True)
untouched_csv = untouched_csv.dropna(how="all").reset_index(drop=True)

In [12]:
#Save this data frame to a Finished csv
#Removes first 24 hrs of data to remove NA's
untouched_csv = untouched_csv[24:]
untouched_csv.to_csv('CSV/CLEAN.csv', index=False)